In [18]:
import pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import math

In [19]:
def get_outlier_indices(df, features):
    outlier_indices = []

    for c in features:
        Q1 = df[c].quantile(0.25)
        Q3 = df[c].quantile(0.75)
        IQR = Q3 - Q1

        outlier_step = 1.5 * IQR

        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)

    outlier_indices = list(set(outlier_indices))
    return outlier_indices

def remove_outliers(df, features):
    outlier_indices = get_outlier_indices(df, features)
    df_cleaned = df.drop(outlier_indices)
    df_cleaned.reset_index(drop=True, inplace=True)
    return df_cleaned

In [20]:
df = pandas.read_csv('Clean_Dataset.csv')
columns_to_drop = ['Unnamed: 0', 'flight']
df = df.drop(columns=columns_to_drop)
le = LabelEncoder()
df.flight_class = df.flight_class.replace({'Economy' : 1,'Business' :2})
df['airline'] = le.fit_transform(df['airline'])
df['source_city'] = le.fit_transform(df['source_city'])
df['destination_city'] = le.fit_transform(df['destination_city'])
df['departure_time'] = le.fit_transform(df['departure_time'])
df['arrival_time'] = le.fit_transform(df['arrival_time'])
df['stops'] = le.fit_transform(df['stops'])
# # One-hot encoding
# columns_to_onehot = ['airline', 'source_city', 'destination_city', 'departure_time', 'arrival_time', 'stops']

# ohe = OneHotEncoder(drop='first', sparse_output=False)
# one_hot_encoded = ohe.fit_transform(df[columns_to_onehot])

# df_encoded = pandas.DataFrame(one_hot_encoded, columns=ohe.get_feature_names_out(columns_to_onehot))
# df = pandas.concat([df.drop(columns=columns_to_onehot), df_encoded], axis=1)
df = remove_outliers(df, ['duration', 'price'])
X = df.drop(columns='price')
y = df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

df_business = df[df['flight_class'] == 2]
X_business = df_business.drop(columns='price')
y_business = df_business.price
X_train_business, X_test_business, y_train_business, y_test_business = train_test_split(X_business, y_business, test_size=0.20, random_state=0)

df_economy = df[df['flight_class'] == 1]
X_economy = df_economy.drop(columns='price')
y_economy = df_economy.price
X_train_economy, X_test_economy, y_train_economy, y_test_economy = train_test_split(X_economy, y_economy, test_size=0.20, random_state=0)



C:\Users\I\AppData\Local\Temp\ipykernel_3796\1441606978.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.flight_class = df.flight_class.replace({'Economy' : 1,'Business' :2})


## Linear regression

In [21]:
lr = LinearRegression().fit(X, y)
y_pred_lr = lr.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_pred_lr)
rmse = math.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred_lr)
print("MSE: " + str(mse))
print("RMSE: " + str(rmse))
print("MAE: " + str(mae))

MSE: 48220128.129044436
RMSE: 6944.071437495761
MAE: 4635.282149073735


## Random forest

In [26]:
# param_grid = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_features': [2, 4, 6, 8],
# }
# rf = RandomForestRegressor(n_jobs=-1)
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
# grid_search.fit(X_train, y_train)
# best_params = grid_search.best_params_
# print(best_params)
final_rf = RandomForestRegressor(n_estimators=300, max_features=8, n_jobs=-1).fit(X_train, y_train)
y_pred_rf = final_rf.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_pred_rf)
rmse = math.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred_rf)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

PicklingError: Could not pickle the task to send it to the workers.

## Multilayer Perceptron

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.metrics import mean_squared_error
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f'Test MAE: {test_mae}')

y_pred = model.predict(X_test_scaled)

# Calculate and print MSE and RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Test MSE: {mse}')
print(f'Test RMSE: {rmse}')

Epoch 1/10
5959/5959 [==============================] - 30s 5ms/step - loss: 127364008.0000 - mae: 6331.6567 - val_loss: 29978430.0000 - val_mae: 3390.8547
Epoch 2/10
5959/5959 [==============================] - 28s 5ms/step - loss: 27978258.0000 - mae: 3246.2256 - val_loss: 27967138.0000 - val_mae: 3203.2166
Epoch 3/10
5959/5959 [==============================] - 28s 5ms/step - loss: 26662324.0000 - mae: 3132.8635 - val_loss: 26969240.0000 - val_mae: 3137.2117
Epoch 4/10
5959/5959 [==============================] - 26s 4ms/step - loss: 25722842.0000 - mae: 3059.3945 - val_loss: 25885694.0000 - val_mae: 3060.2773
Epoch 5/10
5959/5959 [==============================] - 28s 5ms/step - loss: 24728002.0000 - mae: 2989.2683 - val_loss: 25157052.0000 - val_mae: 3026.9038
Epoch 6/10
5959/5959 [==============================] - 29s 5ms/step - loss: 23640984.0000 - mae: 2911.2534 - val_loss: 23561452.0000 - val_mae: 2882.3240
Epoch 7/10
5959/5959 [==============================] - 26s 4ms/step 

## Bagging Regression Tree

In [9]:
base_regressor = DecisionTreeRegressor(random_state=42)

# Create a Bagging Regressor with Decision Trees as base estimators
bagging_regressor = BaggingRegressor(base_regressor, n_estimators=10, random_state=42)

# Define the parameter grid for hyperparameter tuning
# param_grid = {
#     'n_estimators': [5, 10, 15],
#     'max_samples': [0.5, 0.7, 1.0],
#     'max_features': [0.5, 0.7, 1.0]
# }

# # Use GridSearchCV for hyperparameter tuning
# grid_search = GridSearchCV(bagging_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)

# Fit the Bagging Regressor on the training data
bagging_regressor.fit(X_train, y_train)

# Make predictions on the test data
y_pred = bagging_regressor.predict(X_test)

mse = metrics.mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred)
print("MSE: " + str(mse))
print("RMSE: " + str(rmse))
print("MAE: " + str(mae))


KeyboardInterrupt

